In [1]:
from qiskit import IBMQ

In [ ]:
IBMQ.enable_account(token=...)

In [ ]:
import time
from pyqubo import Array
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
import networkx as nx
from networkx.algorithms import bipartite
from random import random
import neal
import itertools
import random
import array as arr
from matplotlib import pyplot as plt

import import_ipynb
import QLS_for_GB1 as QLS

In [4]:
def GB1(G, method='sim_anneal', backend='simulator', subset_size=12):
    
    a=nx.modularity_matrix(G)
    b=nx.to_numpy_array(G)
    

    n=len(G)
    lis=list(range(n))          

    
    liis=lis
    coss=[] #values of energy per node at each round
    communities=[]
    run_time=0
    
    for je in range(5): #tryes to find communities in 5 rounds
        #fits for Zachary Karate Club, in general: k=int(np.sqrt(len(G.nodes())))
        print("Round"+str(je+1))
        energypernode=[]
        stop=False
        if je>0: #removes found communities in each round
            
            for i in range(len(solu)):
                if solu[i] not in liis:
                    #print("Solution node is not in liis anymore!")
                    stop=True #stops if there is no initial improvement in inner loop possible 
                    #--> no update in solu--> nodes of solu already removed in prior round
            if stop:
                break
                
            for i in range(len(solu)):
                liis.remove(solu[i])       
            
        lis=liis
        n=len(lis)
        
        if n==0: #breaking condition in case all communities have been detected --> empty graph
            break
        
        #print("verbleibende Knoten:"+str(n)+str(lis))

        #adjacency/modularity Matrix for current iteration
        c=a[np.ix_(lis,lis)]
        c=(np.asarray(c))
        d=b[np.ix_(lis,lis)]
        d=(np.asarray(d))
        
        
        emax=5 #emax is number of max iterations at each round, not analytically derived, never exceeded in experiments
        eni=0  #energy of previous  step
        it=0
        
        for e in range(emax):        
            
            n=len(lis)
            it+=1
            print("QLS-Iteration: ", it)

            c=a[np.ix_(lis,lis)]
            c=(np.asarray(c))
            d=b[np.ix_(lis,lis)]
            d=(np.asarray(d))
            
            en=0 #energy of best solution       
            
            result=QLS.QLS(c, method, backend, subset_size)
            #print("QLS-Time: ",result[1])
            
            run_time+=result[1]
            en=((result[2])/(2*d.sum())) #"energy density" of current solution --> per definition Newman modularity
            d2=np.array(result[0])
            
            liss=[]
        
            for i in range(n):
                 if(d2[i]==1):#if the spin has value 1, corresponding node is added
                    liss.append(lis[i])
            
            lis=liss
            n=len(lis)
            if n==0:
                break
            ep=en/n      #heuristic criterion, alternatively tested: en --> shows no significant differences
            #print(ep,eni)
            if ep>eni: #update solution and according modularity density
                eni=ep
                solu=np.array(lis)
                
            if ep<eni: #stopping criteria in each round
                coss.append(eni)
                break
          
            energypernode.append(ep)
            #print(energypernode)
            #print(solu) # solu is list of nodes in current community 
            
        communities.append(set(solu))
        
    #print(coss)
    
    total_time=run_time
    rest=[] #classify rest as a single community in case of first stopping criteria mentioned above (no initial improvement)
    for i in (set(G.nodes)-(set().union(*communities))):
        rest.append(i)
    rest=set(rest)
    communities.append(rest) 
    communities=np.unique(np.array(communities))
  
    return communities, total_time #time in milliseconds

##### 